In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [133]:
candidates = [
    {
        "name": "Amy Klobuchar", 
        "endorsements": "https://en.wikipedia.org/wiki/List_of_Amy_Klobuchar_2020_presidential_campaign_endorsements"
    },
    {
        "name": "Joe Biden", 
        "endorsements": "https://en.wikipedia.org/wiki/List_of_Joe_Biden_2020_presidential_campaign_endorsements"
    },
    {
        "name": "Michael Bloomberg", 
        "endorsements": "https://en.wikipedia.org/wiki/List_of_Michael_Bloomberg_2020_presidential_campaign_endorsements"
    },
    {
        "name": "Pete Buttigieg", 
        "endorsements": "https://en.wikipedia.org/wiki/List_of_Pete_Buttigieg_2020_presidential_campaign_endorsements"
    },
    {
        "name": "Bernie Sanders", 
        "endorsements": "https://en.wikipedia.org/wiki/List_of_Bernie_Sanders_2020_presidential_campaign_endorsements"
    },
    {
        "name": "Donald Trump", 
        "endorsements": "https://en.wikipedia.org/wiki/List_of_Donald_Trump_2020_presidential_campaign_endorsements"
    },
    {
        "name": "Elizabeth Warren", 
        "endorsements": "https://en.wikipedia.org/wiki/List_of_Elizabeth_Warren_2020_presidential_campaign_endorsements"
    }
]

In [142]:
filter_out = [
    "Contents", "References", "External links", "Navigation menu", "See also", "Notes"
]

In [171]:
rows = []
for candidate in candidates:
    print(candidate["name"])
    print(candidate["endorsements"])
    html = requests.get(candidate["endorsements"])
    soup = BeautifulSoup(html.text, 'html.parser')
    for h2 in soup.find_all("h2"):
        h2_text = h2.get_text().replace("[edit]","")
        if h2_text not in filter_out:
            elem = h2
            i = 0
            h3_text = False
            h4_text = False
            while True:
                tag = elem.next_sibling.name
                if tag == 'h2':
                    break
                if tag == 'h3':
                    h3_text = elem.next_sibling.get_text().replace("[edit]","")
                if tag == 'h4':
                    h4_text = elem.next_sibling.get_text().replace("[edit]","")
                if tag == 'ul':
                    for li in elem.next_sibling.find_all("li"):
                        try:
                            link = li.find("a")
                            link_text = link.get_text()
                            link_href = "https://en.wikipedia.org" + link["href"]
                            if link_href.find("#cite_note") > -1:
                                link_href = False
                                link_text = li.get_text().replace(link_text,"")
                            rows.append({
                                "name": candidate["name"],
                                "h2": h2_text,
                                "h3": h3_text,
                                "h4": h4_text,
                                "endorser": link_text,
                                "link": link_href
                            })
                        except:
                            pass
                elem = elem.next_sibling
                i += 1

Amy Klobuchar
https://en.wikipedia.org/wiki/List_of_Amy_Klobuchar_2020_presidential_campaign_endorsements
Joe Biden
https://en.wikipedia.org/wiki/List_of_Joe_Biden_2020_presidential_campaign_endorsements
Michael Bloomberg
https://en.wikipedia.org/wiki/List_of_Michael_Bloomberg_2020_presidential_campaign_endorsements
Pete Buttigieg
https://en.wikipedia.org/wiki/List_of_Pete_Buttigieg_2020_presidential_campaign_endorsements
Bernie Sanders
https://en.wikipedia.org/wiki/List_of_Bernie_Sanders_2020_presidential_campaign_endorsements
Donald Trump
https://en.wikipedia.org/wiki/List_of_Donald_Trump_2020_presidential_campaign_endorsements
Elizabeth Warren
https://en.wikipedia.org/wiki/List_of_Elizabeth_Warren_2020_presidential_campaign_endorsements


In [172]:
df = pd.DataFrame(rows)
df.head()

,name,h2,h3,h4,endorser,link
0,Amy Klobuchar,Federal officials,Vice Presidents,Former,Walter Mondale,https://en.wikipedia.org/wiki/Walter_Mondale
1,Amy Klobuchar,Federal officials,U.S. Senators,Current,Tina Smith,https://en.wikipedia.org/wiki/Tina_Smith
2,Amy Klobuchar,Federal officials,U.S. Representatives,Current,Linda Sánchez,https://en.wikipedia.org/wiki/Linda_S%C3%A1nchez
3,Amy Klobuchar,Federal officials,U.S. Representatives,Current,Angie Craig,https://en.wikipedia.org/wiki/Angie_Craig
4,Amy Klobuchar,Federal officials,U.S. Representatives,Current,Dean Phillips,https://en.wikipedia.org/wiki/Dean_Phillips


In [173]:
df.to_csv("endorsements.csv", index=False)